<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/polyhedron-gdl/advanced-machine-learning-for-finance/blob/main/2022/1-notebooks/chapter-4-2.ipynb">
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Text Vectorization

In [1]:
import re
import nltk
import numpy  as np
import pandas as pd

## What are Word Vectorization and Why It is so Important

Word vectorization generically refers to techniques used to convert text into numbers. There may be different numerical representations of the same text. 

Many Machine Learning algorithms and almost all Deep Learning Architectures are incapable of processing *strings* or *plain text* in their raw form. They require numbers as inputs to perform any sort of job, be it classification, regression, etc. in broad terms.


## Bag-of-Words (BOW)

As we have already said, in order to perform machine learning on text, we need to transform our documents into vector representations such that we can apply numeric machine learning. This process is called feature extraction or more simply, vectorization.

We will explore several choices, each of which extends or modifies the base bag-of-words model to describe semantic space. We will look at four types of vector encoding - frequency, one-hot, TF-IDF.

### Sample corpus of text documents

<table><tr>
<td> <img align="left" width="100" height="100" src="./pic/text_analytics_with_python.jpg"/> </td>
<td> The following examples are taken from "Text Analytics with Python" by Dipanjan Sarkar (Apress, 2019).     </td>
</tr></table>

[Here](https://github.com/Apress/text-analytics-w-python-2e) you can find the original notebook used in the aforementioned textbook.

In [2]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, e...",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beaut...,weather
7,The dog is lazy but the brown fox is quick!,animals


### Simple text pre-processing

In [3]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

**Regular Expression Flags**

**re.I - 
re.IGNORECASE**

Perform case-insensitive matching; expressions like [A-Z] will also match lowercase letters. Full Unicode matching (such as Ü matching ü) also works unless the re.ASCII flag is used to disable non-ASCII matches. The current locale does not change the effect of this flag unless the re.LOCALE flag is also used. Corresponds to the inline flag (?i).

**re.A - 
re.ASCII**

Make \\w, \\W, \\b, \\B, \\d, \\D, \\s and \\S perform ASCII-only matching instead of full Unicode matching. This is only meaningful for Unicode patterns, and is ignored for byte patterns. Corresponds to the inline flag (?a).

In [4]:
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham bacon eggs toast beans',
       'love green eggs ham sausages bacon',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'], dtype='<U51')

### Frequency Vectors

The simplest vector encoding model is to simply fill in the vector with the frequency of each word as it appears in the document;
In this encoding scheme each document is represented as the multiset of the tokens that compose it and the value for each word position in the vectr is its count;
This representation can either be a straight count encoding or a normalized encoding where each word is weighted by the total number of words in the document.

In **Scikit-Learn** The CountVectorizer transformer from the sklearn.feature_extraction model has its own internal tokenization and normalization methods. The fit method of the vectorizer expects an iterable or list of strings or file objects, and creates a dictionary of the vocabulary on the corpus. When transform is called, each individual document is transformed into a sparse array whose index tuple is the row (the document ID) and the token ID from the dictionary, and whose value is the count.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
# get bag of words features in sparse format
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix

<8x20 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [6]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]],
      dtype=int64)

In [8]:
# get all unique words in the corpus
vocab = cv.get_feature_names_out()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,0,0,0
3,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0
5,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1
7,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0



### One-Hot Encoding

Because they disregard grammar and the relative position of words in documents, frequency-based encoding methods suffer from the long tail, or Zipfian distribution, that characterizes natural language. As a result, tokens that occur very frequently are orders of magnitude more “significant” than other, less frequent ones. This can have a significant impact on some models (e.g., generalized linear models) that expect normally distributed features. 

A solution to this problem is **one-hot encoding**, a boolean vector encoding method that marks a particular vector index with a value of true (1) if the token exists in the document and false (0) if it does not. In other words, each element of a one-hot encoded vector reflects either the presence or absence of the token in the described text.

In [9]:
# get bag of words features in sparse format
cv = CountVectorizer(min_df=0., max_df=1., binary=True)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix

<8x20 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [10]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]],
      dtype=int64)



**TF - Term Frequency**: Just counting the number of words in each document has 1 issue: it will give more weightage to longer documents than shorter documents. To avoid this, we can use frequency i.e. #count(word) / #Total words, in each document.


### Term Frequency-Inverse Document Frequency

The bag-of-words representations that we have explored so far only describe a document in a standalone fashion, not taking into account the context of the corpus. 
A better approach would be to consider the relative frequency or rareness of tokens in the document against their frequency in other documents. The central insight is that meaning is most likely encoded in the more rare terms from a document.

TF-IDF, term frequency-inverse document frequency, encoding normalizes the frequency of tokens in a document with respect to the rest of the corpus. 
This encoding approach accentuates terms that are very relevant to a specific instance, as shown in Figure, where the token studio has a higher relevance to this document since it only appears there.

The inverse document frequency is a measure of how much information the word provides, that is, whether the term is common or rare across all documents. 
It is the logarithmically scaled inverse fraction of the documents that contain the word, obtained by dividing the total number of documents by the number of documents containing the term, and then taking the logarithm of that quotient: 

\begin{equation}idf(t,D) = \log \frac{N}{\vert \{  d \in D: t \in d\}\vert} \end{equation}  

where the numerator ($N$) is the total number of documents in the corpus and the denominator is the number of documents where the term $t$ appears.

Then tf-idf is calculated as: 

\begin{equation} tfidf(t,d,D) = tf(t,d) \cdot idf(t,D)\end{equation}

A high weight in tf-idf is reached by a high term frequency (in the given document) and a low document frequency of the term in the whole collection of documents; the weights hence tend to filter out common terms.  Since the ratio inside the idf's log function is always greater than or equal to 1, the value of idf (and tf-idf) is greater than or equal to 0. As a term appears in more documents, the ratio inside the logarithm approaches 1, bringing the idf and tf-idf closer to 0.

![image.png](./pic/tf-idf-0.png)

A simple example might serve to explain the structure of the TDM more clearly. Assume we have a simple corpus consisting of two documents, Doc1 and Doc2, with the following content:

Doc1 = "I like databases"
Doc2 = "I dislike databases",

then the document-term matrix would be:

![image.png](./pic/tf-idf-1.png)

Clearly there is nothing special about rows and columns – we could just as easily transpose them. If we did so, we’d get a term document matrix (TDM) in which the terms are rows and documents columns. One can work with either a DTM or TDM. Using the raw count of a term in a document, i.e. the number of times that term t occurs in document d, is the simplest choice to measure the term frequency $tf(t,d)$. If we denote the raw count by $f_{t,d}$, then the simplest $tf$ scheme is $tf(t,d) = f_{t,d}$. Other possibilities include

- Boolean "frequencies": $tf(t,d) = 1$ if $t$ occurs in $d$ and $0$ otherwise;
- Term frequency adjusted for document length : $f_{t,d} \big/ \text{(number of words in d)}$;
- Logarithmically scaled frequency: $tf(t,d) = \log (1 + f_{t,d})$;
- Augmented frequency, to prevent a bias towards longer documents, e.g. raw frequency divided by the raw frequency of the most occurring term in the document.

![image.png](./pic/3_1_text_vectorization_pic_6.png)

Scikit-Learn provides a transformer called the TfidfVectorizer in the module called **feature_extraction.text** for vectorizing documents with TF–IDF scores. Under the hood, the TfidfVectorizer uses the CountVectorizer estimator we used to produce the bag-of-words encoding to count occurrences of tokens, followed by a TfidfTransformer, which normalizes these occurrence counts by the inverse document frequency. 

The input for a TfidfVectorizer is expected to be a sequence of filenames, file-like objects, or strings that contain a collection of raw documents, similar to that of the CountVectorizer. As a result, a default tokenization and preprocessing method is applied unless other functions are specified. The vectorizer returns a sparse matrix representation in the form of ((doc, term), tfidf) where each key is a document and term pair and the value is the TF–IDF score.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., norm='l2',
                     use_idf=True, smooth_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names_out()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0.00,0.00,0.60,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00,0.0
1,0.00,0.00,0.49,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.49,0.00,0.0
2,0.00,0.00,0.00,0.00,0.00,0.38,0.38,0.00,0.38,0.00,0.00,0.53,0.00,0.38,0.00,0.38,0.00,0.00,0.00,0.0
3,0.32,0.38,0.00,0.00,0.38,0.00,0.00,0.32,0.00,0.00,0.32,0.00,0.38,0.00,0.00,0.00,0.32,0.00,0.38,0.0
4,0.39,0.00,0.00,0.00,0.00,0.00,0.00,0.39,0.00,0.47,0.39,0.00,0.00,0.00,0.39,0.00,0.39,0.00,0.00,0.0
5,0.00,0.00,0.00,0.37,0.00,0.42,0.42,0.00,0.42,0.00,0.00,0.00,0.00,0.42,0.00,0.42,0.00,0.00,0.00,0.0
6,0.00,0.00,0.36,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,0.00,0.5
7,0.00,0.00,0.00,0.00,0.00,0.45,0.45,0.00,0.45,0.00,0.00,0.00,0.00,0.45,0.00,0.45,0.00,0.00,0.00,0.0


### Extracting Features for New Documents

Suppose you built a machine learning model to classify and categorize news articles and it is currently in production. How you can generate features for completely new documents so that you can feed it into the machine learning models for prediction? The Scikit-Learn API provides the `transform` function for the vectorizers we discussed previously and we can leverage it to ge features for a completely new document that was not present in our corpus. For example:

In [12]:
new_doc = 'the sky is green today'

pd.DataFrame(np.round(tv.transform([new_doc]).toarray(), 2), 
             columns=tv.get_feature_names_out())

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.46,0.0,0.63


## Document Similarity

When you have vectorized your text, we can try to define a distance metric such that documents that are closer together in feature space are more similar. There are a number of different measures that can be used to determine document similarity. Fundamentally, each relies on our ability to imagine documents as points in space, where the relative closeness of any two documents is a measure of their similarity.

We can measure vector similarity with cosine distance, using the cosine of the angle between the two vectors to assess the degree to which they share the same orientation. In effect, the more parallel any two vectors are, the more similar the documents will be (regardless of their magnitude).

<!--
<div>
<img src="./img/3_1_text_vectorization_pic_9.png" width="500"/>
</div>
-->
![pic](./pic/3_1_text_vectorization_pic_9.png)

*(image source: Bengfort B. et al. "Text Analysis with Python")*

Mathematically, Cosine similarity metric measures the cosine of the angle between two n-dimensional vectors projected in a multi-dimensional space. The Cosine similarity of two documents will range from 0 to 1. If the Cosine similarity score is 1, it means two vectors have the same orientation. The value closer to 0 indicates that the two documents have less similarity.

The mathematical equation of Cosine similarity between two non-zero vectors is: 

\begin{equation} \text{similarity} = \cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\left\Vert\mathbf{A}\right\Vert \,\left\Vert\mathbf{B}\right\Vert} = \frac{\sum\limits_{i=1}^n A_iB_i}{\sqrt{\sum\limits_{i=1}^n A_i^2} \sqrt{ \sum\limits_{i=1}^n B_i^2}} \end{equation}

Let’s see an example of how to calculate the cosine similarity between two text document. The common way to compute the Cosine similarity is to first we need to count the word occurrence in each document. To count the word occurrence in each document, we can use **CountVectorizer** or **TfidfVectorizer** functions that are provided by Scikit-Learn library.

doc_1 = "Data is the oil of the digital economy" 
doc_2 = "Data is a new oil" 

and consider the following frequency matrix

<!--
<img src='./img/tf-idf-2.png'>
-->
![image.png](./pic/tf-idf-2.png)

We can write

In [13]:
doc_1 = "Data is the oil of the digital economy" 
doc_2 = "Data is a new oil" 
data = [doc_1, doc_2]

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
vector_matrix = count_vectorizer.fit_transform(data)

Here, is the unique tokens list found in the data.

In [15]:
tokens = count_vectorizer.get_feature_names_out()
tokens

array(['data', 'digital', 'economy', 'is', 'new', 'of', 'oil', 'the'],
      dtype=object)

In [16]:
vector_matrix.toarray()

array([[1, 1, 1, 1, 0, 1, 1, 2],
       [1, 0, 0, 1, 1, 0, 1, 0]], dtype=int64)

In [17]:
def create_dataframe(matrix, tokens):

    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

create_dataframe(vector_matrix.toarray(),tokens)

,data,digital,economy,is,new,of,oil,the
doc_1,1,1,1,1,0,1,1,2
doc_2,1,0,0,1,1,0,1,0


In [18]:
a = vector_matrix.toarray()
v1 = np.array(a[0])
v2 = np.array(a[1])

In [19]:
n1 = np.linalg.norm(v1)
n2 = np.linalg.norm(v2)
prod = np.inner(v1,v2)

In [20]:
print(prod/(n1*n2))

0.4743416490252569


In [21]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['doc_1','doc_2'])

,doc_1,doc_2
doc_1,1.000000,0.474342
doc_2,0.474342,1.000000


Let’s check the cosine similarity with `TfidfVectorizer`, and see how it change over `CountVectorizer`.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidf_vect = TfidfVectorizer()
vector_matrix = Tfidf_vect.fit_transform(data)

tokens = Tfidf_vect.get_feature_names_out()
create_dataframe(vector_matrix.toarray(),tokens)

,data,digital,economy,is,new,of,oil,the
doc_1,0.243777,0.34262,0.34262,0.243777,0.000000,0.34262,0.243777,0.68524
doc_2,0.448321,0.00000,0.00000,0.448321,0.630099,0.00000,0.448321,0.00000


In [23]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['doc_1','doc_2'])

,doc_1,doc_2
doc_1,1.000000,0.327871
doc_2,0.327871,1.000000


Here, using `TfidfVectorizer` we get the cosine similarity between doc_1 and doc_2 is 0.32.  Where the `CountVectorizer` has returned the cosine similarity of doc_1 and doc_2 is 0.47. This is because `TfidfVectorizer` penalized the most frequent words in the document such as stopwords.

Now, the **distance** can be defined as 

\begin{equation}
d =1-\mathrm{CosineSimilarity} 
\end{equation}

The intuition behind this is that if 2 vectors are perfectly the same then similarity is 1 (angle=0) and thus, distance is 0 (1-1=0).


Let's apply the same analysis to our toy corpus:

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7
0,1.000000,0.820599,0.000000,0.000000,0.000000,0.192353,0.817246,0.000000
1,0.820599,1.000000,0.000000,0.000000,0.225489,0.157845,0.670631,0.000000
2,0.000000,0.000000,1.000000,0.000000,0.000000,0.791821,0.000000,0.850516
3,0.000000,0.000000,0.000000,1.000000,0.506866,0.000000,0.000000,0.000000
4,0.000000,0.225489,0.000000,0.506866,1.000000,0.000000,0.000000,0.000000
5,0.192353,0.157845,0.791821,0.000000,0.000000,1.000000,0.115488,0.930989
6,0.817246,0.670631,0.000000,0.000000,0.000000,0.115488,1.000000,0.000000
7,0.000000,0.000000,0.850516,0.000000,0.000000,0.930989,0.000000,1.000000


## Word Embedding

Humans have always excelled at understanding languages. It is easy for humans to understand the relationship between words but for computers, this task may not be simple. Word embeddings are basically a form of word representation that bridges the human understanding of language to that of a machine. For example, we humans understand the words like king and queen, man and woman, tiger and tigress have a certain type of relation between them but how can a computer figure this out?

The different encoding we have discussed so far is arbitrary as it does not capture any relationship between words. 
It can be challenging for a model to interpret, for example, a linear classifier learns a single weight for each feature. 
Because there is no relationship between the similarity of any two words and the similarity of their encodings, this feature-weight combination is not meaningful.

It should be nice to have representations of text in an n-dimensional space where words that have the same meaning have a similar representation. Meaning that two similar words are represented by almost similar vectors that are very closely placed in a vector space. 

Thus when using word embeddings, all individual words are represented as real-valued vectors in a predefined vector space. 
Each word is mapped to one vector and the vector values are learned in a way that resembles a neural network. Word2Vec is one of the most popular technique to learn word embeddings using shallow neural network. It was developed by Tomas Mikolov in 2013 at Google.

The concept of embeddings arises from a branch of Natural Language Processing called - “Distributional Semantics”. It is based on the simple intuition that: ***Words that occur in similar contexts tend to have similar meanings***. In other words, a word’s meaning is given by the words that it appears frequently with.

## Word2Vec

Word2vec is a method to efficiently create word embeddings by using a two-layer neural network.  It was developed by Tomas Mikolov, et al. at Google in 2013 as a response to make the neural-network-based training of the embedding more efficient and since then has become the de facto standard for developing pre-trained word embedding. 

The input of word2vec is a text corpus and its output is a set of vectors known as feature vectors that represent words in that corpus. The Word2Vec objective function causes the words that have a similar context to have similar embeddings. 
Thus in this vector space, these words are really close. Mathematically, the cosine of the angle (Q) between such vectors should be close to 1, i.e. angle close to 0.

Word2vec is not a single algorithm but a combination of two techniques - **CBOW(Continuous bag of words)** and **Skip-gram** model. Both these are shallow neural networks which map word(s) to the target variable which is also a word(s). Both these techniques learn weights which act as word vector representations. 



### CBOW Architecture 

CBOW predicts the probability of a word to occur **given the words surrounding it**. We can consider a single word or a group of words. But for simplicity, we will take a single context word and try to predict a single target word.
The English language contains almost 1.2 million words, making it impossible to include so many words in our example. So we will consider a small example in which we have only four words i.e. ***live***, ***home***, ***they*** and ***at***. For simplicity, we will consider that the corpus contains only one sentence, that being, ***They live at home***. First, we convert each word into a one-hot encoding form. Also, we'll not consider all the words in the sentence but ll only take certain words that are in a window.

![image.png](./pic/word-embedding-1.png)

For example for a window size equal to three, we only consider three words
in a sentence. The middle word is to be predicted and the surrounding two
words are fed into the neural network as context. The window is then slid
and the process is repeated again. Finally, after training the network repeatedly by sliding the window a
shown above, we get weights which we use to get the embeddings as
shown below. Usually, we take a window size of around 8-10 words and have a
vector size of 300.

![image.png](./pic/word-embedding-2.png)

### Skip-gram model

The Skip-gram model architecture usually tries to achieve the reverse of what the CBOW model does. It tries to predict the source context words (surrounding words) given a target word (the centre word)

The working of the skip-gram model is quite similar to the CBOW but there is just a difference in the architecture of its neural network and the way the weight matrix is generated  as shown in the figure below:

![image.png](./pic/3_1_text_vectorization_pic_16.png)

So now which one of the two algorithms should we use for implementing word2vec? Turns out for large corpus with higher dimensions, it is better to use skip-gram but is slow to train. Whereas CBOW is better for small corpus and is faster to train too.

## Implementing a word2vec model using a CBOW NN architecture

### Load up sample corpus

In [25]:
import nltk
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [26]:
from nltk.corpus import gutenberg
book = gutenberg.sents('carroll-alice.txt') 

In [27]:
from string import punctuation

remove_terms = punctuation + '0123456789'

norm_book = [[word.lower() for word in sent if word not in remove_terms] for sent in book]
norm_book = [' '.join(tok_sent) for tok_sent in norm_book]
norm_book = filter(None, normalize_corpus(norm_book))
norm_book = [tok_sent for tok_sent in norm_book if len(tok_sent.split()) > 2]

print('Total lines:', len(book))
print('\nSample line:', book[10])

print('Total lines:', len(norm_book))
print('\nProcessed line:', norm_book[10])

Total lines: 1703

Sample line: ['The', 'rabbit', '-', 'hole', 'went', 'straight', 'on', 'like', 'a', 'tunnel', 'for', 'some', 'way', ',', 'and', 'then', 'dipped', 'suddenly', 'down', ',', 'so', 'suddenly', 'that', 'Alice', 'had', 'not', 'a', 'moment', 'to', 'think', 'about', 'stopping', 'herself', 'before', 'she', 'found', 'herself', 'falling', 'down', 'a', 'very', 'deep', 'well', '.']
Total lines: 1290

Processed line: thought alice fall shall think nothing tumbling stairs


### Build Vocabulary

**What is Padding***

Padding comes from the need to encode sequence data into contiguous batches: in order to make all sequences in a batch fit a given standard length, it is necessary to pad or truncate some sequences.

Let's take a close look.

When processing sequence data, it is very common for individual samples to have different lengths. Consider the following example (text tokenized as words):

In [28]:
example = [
  ["Hello", "world", "!"],
  ["How", "are", "you", "doing", "today"],
  ["The", "weather", "will", "be", "nice", "tomorrow"],
]

After vocabulary lookup, the data might be vectorized as integers, e.g.:

In [29]:
example = [
  [71, 1331, 4231],
  [73, 8, 3215, 55, 927], 
  [83, 91, 1, 645, 1253, 927],
]

The data is a nested list where individual samples have length 3, 5, and 6, respectively. **Since the input data for a deep learning model must be a single tensor** (of shape e.g. (batch_size, 6, vocab_size) in this case), **samples that are shorter than the longest item need to be padded with some placeholder value** (alternatively, one might also truncate long samples before padding short samples).

Keras provides a utility function to truncate and pad Python lists to a common length: `tf.keras.preprocessing.sequence.pad_sequences`.

In [30]:
import keras 

raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# By default, this will pad using 0s; it is configurable via the
# "value" parameter.
# Note that you could "pre" padding (at the beginning) or
# "post" padding (at the end).
# We recommend using "post" padding when working with RNN layers
# (in order to be able to use the
# CuDNN implementation of the layers).
padded_inputs = keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


`fit_on_texts` Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So ***lower integer means more frequent word*** (often the first few are stop words because they appear a lot).

In [31]:
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils         import np_utils

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_book)
word2id = tokenizer.word_index

In [32]:
#
# in this case we explicity indicate that the placeholder to be used for padding is 0 (this is not strictly necessary)
#
word2id['PAD'] = 0

`texts_to_word_sequences` Transforms each text in a sequence of word. 

In [33]:
wids = [[word2id[w] for w in text.text_to_word_sequence(row)] for row in norm_book]

In [34]:
wids[0]

[2, 337, 669, 1273, 1274]

We can make also a very simple mapping to reverse the integer coding and recover the word from the id

In [35]:
id2word = {v:k for k, v in word2id.items()}
for i in range(1,10):
    print(i, id2word[i])

1 said
2 alice
3 little
4 one
5 would
6 know
7 went
8 like
9 could


In [36]:
for w in wids[0]:
    print(id2word[w])

alice
adventures
wonderland
lewis
carroll


In [37]:
vocab_size = len(word2id)
embed_size = 100
window_size = 2

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 2379
Vocabulary Sample: [('said', 1), ('alice', 2), ('little', 3), ('one', 4), ('would', 5), ('know', 6), ('went', 7), ('like', 8), ('could', 9), ('time', 10)]


### Build (context_words, target_word) pair generator

In [38]:
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word    = []            
            start         = index - window_size
            end           = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = sequence.pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)

In [39]:
# Rember that argwhere find the indices of array elements that are non-zero, grouped by element.
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

Context (X): ['alice', 'adventures', 'lewis', 'carroll'] -> Target (Y): wonderland
Context (X): ['alice', 'beginning', 'tired', 'sitting'] -> Target (Y): get
Context (X): ['beginning', 'get', 'sitting', 'sister'] -> Target (Y): tired
Context (X): ['get', 'tired', 'sister', 'bank'] -> Target (Y): sitting
Context (X): ['tired', 'sitting', 'bank', 'nothing'] -> Target (Y): sister
Context (X): ['sitting', 'sister', 'nothing', 'twice'] -> Target (Y): bank
Context (X): ['sister', 'bank', 'twice', 'peeped'] -> Target (Y): nothing
Context (X): ['bank', 'nothing', 'peeped', 'book'] -> Target (Y): twice
Context (X): ['nothing', 'twice', 'book', 'sister'] -> Target (Y): peeped
Context (X): ['twice', 'peeped', 'sister', 'reading'] -> Target (Y): book
Context (X): ['peeped', 'book', 'reading', 'pictures'] -> Target (Y): sister


### Build CBOW Deep Network Model

**What is a "backend"?**

Keras is a model-level library, providing high-level building blocks for developing deep learning models. It does not handle itself low-level operations such as tensor products, convolutions and so on. Instead, it relies on a specialized, well-optimized tensor manipulation library to do so, serving as the "backend engine" of Keras. Rather than picking one single tensor library and making the implementation of Keras tied to that library, Keras handles the problem in a modular way, and several different backend engines can be plugged seamlessly into Keras. At this time, Keras has two backend implementations available: the TensorFlow backend and the Theano backend.


**The Embedding Layer**

The Embedding layer is defined as the first hidden layer of a network. It must specify 3 arguments:

- **input_dim**: This is the size of the vocabulary in the text data. For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.

- **output_dim**: This is the size of the vector space in which words will be embedded. It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.

- **input_length**: This is the length of input sequences, as you would define for any input layer of a Keras model. For example, if all of your input documents are comprised of 1000 words, this would be 1000.

**Lambda Layer**

The Lambda layer exists so that arbitrary expressions can be used as a Layer when constructing Sequential and Functional API models. Lambda layers are best suited for simple operations or quick experimentation

In [40]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))

cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print(cbow.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 100)            237900    
_________________________________________________________________
lambda (Lambda)              (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2379)              240279    
Total params: 478,179
Trainable params: 478,179
Non-trainable params: 0
_________________________________________________________________
None


### Train model for 5 epochs

**What is `train_on_batch`**

`train_on_batch` allows you to expressly update weights based on a collection of samples you provide, without regard to any fixed batch size. You would use this in cases when that is what you want: to train on an explicit collection of samples. You could use that approach to maintain your own iteration over multiple batches of a traditional training set but allowing fit or fit_generator to iterate batches for you is likely simpler.

One case when it might be nice to use `train_on_batch` is for updating a pre-trained model on a single new batch of samples. Suppose you've already trained and deployed a model, and sometime later you've received a new set of training samples previously never used. You could use `train_on_batch` to directly update the existing model only on those samples. Other methods can do this too, but it is rather explicit to use train_on_batch for this case.

In [45]:
for epoch in range(1, 15):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 10000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Processed 10000 (context, word) pairs
Epoch: 1 	Loss: 98396.26991511881

Processed 10000 (context, word) pairs
Epoch: 2 	Loss: 98768.56974580139

Processed 10000 (context, word) pairs
Epoch: 3 	Loss: 98866.98952811956



KeyboardInterrupt: 

### Get word embeddings

In [42]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(2378, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
alice,0.351183,0.041107,0.090196,-0.215083,0.170995,-0.345772,-0.119684,0.068980,-0.180764,0.051785,...,-0.233945,0.513039,0.169514,0.510741,-0.673383,-0.098482,-0.123255,0.428761,0.035849,0.300336
little,-0.188662,0.003105,-0.108142,-0.774432,-0.117824,0.101573,0.407862,0.025299,0.266874,-0.006119,...,0.181358,-0.111602,0.065270,-0.277129,0.226172,-0.273504,0.074520,-0.157811,-0.151565,-0.057601
one,0.666672,-0.358681,0.191382,-0.675251,-0.274691,-0.153004,0.089441,-0.514175,0.083103,0.226477,...,-0.024816,0.182160,0.162045,0.405100,-0.020502,0.038454,0.313101,0.192715,-0.285474,0.370692
would,0.321406,0.312559,0.088120,-0.499484,-0.450157,-0.186578,-0.084536,-0.374199,-0.205349,0.082250,...,0.083769,-0.178289,0.173287,-0.119762,0.374727,0.103557,0.187335,-0.063804,0.030449,0.244675
know,0.036590,0.041059,0.210075,-0.268672,-0.176965,0.097156,-0.027579,-0.100951,-0.091927,-0.160180,...,-0.145163,0.241729,-0.281267,0.135270,0.204009,-0.048631,0.183325,-0.234910,0.003380,-0.168523


### Build a distance matrix to view the most similar words (contextually)

In [44]:
from sklearn.metrics.pairwise import euclidean_distances

# compute pairwise distance matrix
distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

# view contextually similar words
similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['cat', 'caterpillar', 'hatter', 'queen']}

similar_words

(2378, 2378)


{'cat': ['cheshire', 'mad', 'pig', 'pointing', 'sneeze'],
 'caterpillar': ['hookah', 'exactly', 'mushroom', 'held', 'puppy'],
 'hatter': ['bread', 'butter', 'officers', 'dormouse', 'write'],
 'queen': ['three', 'behind', 'hedgehog', 'duchess', 'spoke'],
 'flamingo': ['fight', 'executions', 'music', 'roses', 'drew']}

## Pre-Trained Embedding Models

Although in real applications we train our model over Wikipedia text with a window size around 5- 10. 
The number of words in the corpus is around 13 million, hence it takes a huge amount of time and resources to generate these embeddings. 
To avoid this we can use the pre-trained word vectors that are already trained and we can easily use them. 
Here are the links to download pre-trained Word2Vec or GloVe.

# Appendix

## What is the meaning of the yield instruction in Python?

To understand what yield does, you must understand what generators are. And before you can understand generators, you must understand iterables.

**Iterables**

When you create a list, you can read its items one by one. Reading its items one by one is called iteration. `mylist` is an iterable. When you use a list comprehension, you create a list, and so an iterable:

In [ ]:
mylist = [x*x for x in range(3)]
for i in mylist:
    print(i)

In [ ]:
for i in mylist:
    print(i)

Everything you can use "for... in..." on is an iterable; lists, strings, files...

These iterables are handy because you can read them as much as you wish, but **you store all the values in memory** and this is not always what you want when you have a lot of values.

**Generators**

Generators are iterators, a kind of iterable you can only iterate over once. Generators **do not store all the values in memory, they generate the values on the fly**:

In [ ]:
mygenerator = (x*x for x in range(3))
for i in mygenerator:
    print(i)

It is just the same except you used () instead of []. BUT, you cannot perform `for i in mygenerator` a second time since generators can only be used once: they calculate 0, then forget about it and calculate 1, and end calculating 4, one by one:

In [ ]:
for i in mygenerator:
    print(i)

**Yield**

yield is a keyword that is used like return, except ***the function will return a generator***.

In [ ]:
def create_generator():
    mylist = range(10)
    for i in mylist:
        yield i*i

mygenerator = create_generator() # create a generator
print(mygenerator) # mygenerator is an object!

for i in mygenerator:
    print(i)

# References and Credits

***Bird S. et al.***, "*Natural Language Processing with Python*" O'Reilly (2009)

***Bengfort B. et al.***, "*Applied Text Analysis with Python*" O'Reilly (2018)

***Sarkar D.***, "*Text Analytics with Python A Practitioner's Guide to Natural Language Processing 2nd Ed*" APress (2019)

***Pennington J. et al.***, "*GloVe: Global Vectors for Word Representation*", [pdf](https://nlp.stanford.edu/pubs/glove.pdf)

***Mikolov T. et al.***, "*Efficient Estimation of Word Representations in Vector Space*", arXiv:1301.3781v3  7 Sep 2013 [pdf](https://arxiv.org/abs/1301.3781)